<div style="text-align: center"><h1><font size='10' face = "Comic sans MS" color="##75ccbc">EDA for a Healthy GI Tract <br> ⚕️💪🏽</font></h1></div>

In [ ]:
import os
from typing import List, Tuple
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from skimage import io
from skimage.color import gray2rgb
from skimage.transform import resize
from rich.jupyter import print


seed = 400  # to make the notebook reproducible (including images)

# Metadata 📄

In [ ]:
train_df = pd.read_csv("/kaggle/input/uw-madison-gi-tract-image-segmentation/train.csv")
train_df.describe(include="all")

# Create new metadata 🧪

It is clear that the slice id are repeating. There is one entry for each class and it's segmentation. Lets flatten this to make it easier to work with. 

The idea is to make a metadata with a mapping from an image to all its attributes. This would make data loading much easier! 

In [ ]:
def create_new_metadata(train_df):
    """Create new metadata with a single row for single image."""
    train_path = Path("/kaggle/input/uw-madison-gi-tract-image-segmentation/train/")
    save_path = Path("/kaggle/working/")
    
    column_names = ["id", "slice_h", "slice_w","pixel_h", "pixel_w", "image_file", 
                "large_bowel", "small_bowel", "stomach"]
    new_df = pd.DataFrame(columns = column_names)
    
    for case in os.listdir(train_path):
        for case_day in os.listdir(train_path/case):
            for slice_file in os.listdir(train_path/case/case_day/"scans"):
                # Create single entry for unqiue slice/image.
                curr_slice = {}
                curr_slice["image_file"] = slice_file

                # Expand id and resolution info from slice file name
                name_splits = slice_file.split(".png")[0].split("_")
                curr_slice["id"] = case_day + "_" + "_".join(name_splits[:2])
                for k, v in zip(["slice_h", "slice_w","pixel_h", "pixel_w"], name_splits[-4:]):
                    curr_slice[k] = v

                # Use id to fetch instances from train_df
                slice_info = train_df[train_df['id'] == curr_slice["id"]]

                # Save segmentations to columns with their class name
                for x in ["large_bowel", "small_bowel", "stomach"]:
                    curr_slice[x] = slice_info[slice_info["class"] == x]["segmentation"].values[0]

                new_df = new_df.append(curr_slice, ignore_index=True)
    new_df.to_csv(save_path/"train_flat.csv", index=False)

# _ = create_new_metadata(train_df)

#### Note: The output of the above method is made available here - 
https://www.kaggle.com/datasets/bsridatta/uwmadison-flattened-metadata

In [ ]:
# load flattened metadata
train_df = pd.read_csv("/kaggle/input/uwmadison-flattened-metadata/train_flat.csv")
train_path = Path("/kaggle/input/uw-madison-gi-tract-image-segmentation/train/")

# Analyze Metadata 📈

In [ ]:
train_df.describe(include="all")

# Temporal spread of scans for cases ⏰

In [ ]:
time_info = {}
for case in os.listdir(train_path):
    time_info[case] = len(os.listdir(train_path/case))
df = pd.DataFrame({"case": time_info.keys(), "days": time_info.values()})
fig = px.histogram(df, x='case', y='days', color_discrete_sequence=['#FF6692'], title='Number of days ases').update_xaxes(categoryorder='total descending')
fig.show()


# Quick stats ⭐️ 

In [ ]:
print(f"[blue]🌟 Value count - Slice_h {dict(train_df.slice_h.value_counts())}")
print(f"[blue]🌟 Value count - Slice_w {dict(train_df.slice_w.value_counts())}")
print(f"[blue]🌟 Value count - pixel_h {dict(train_df.pixel_h.value_counts())}")
print(f"[blue]🌟 Value count - pixel_w {dict(train_df.pixel_w.value_counts())}")

Pixel resolution is same for all images

In [ ]:
def get_image_array(file_paths: List[str], 
                    image_folder:str="/kaggle/input/uw-madison-gi-tract-image-segmentation/train/", 
                    shape: Tuple=(300,300)
                   ) -> np.array:
    """read all images as a single array for plotly."""
    read_img = lambda file: io.imread(f"{image_folder}/{file}")
    images = []
    empty = 0
    for x in file_paths:
        img = read_img(x)
        images.append(resize(img, shape))
        if not img.any():
            empty += 1
    # print(f"{empty} images are blank")
    return np.asarray(images)

def parse_paths(ids: List[str], image_files: List[str]) -> List[str]:
    paths = []
    for id, file in zip(ids, image_files):
        splits = id.split("_")
        paths.append(f"{splits[0]}/{splits[0]}_{splits[1]}/scans/{file}")
    return paths

# Sample Images 🕵️‍♂️

In [ ]:
save_path = Path("/kaggle/working/")

samples = train_df[['id', 'image_file']].sample(5, random_state=seed)
sample_paths = parse_paths(samples.id.values, samples.image_file.values)
sample_images = get_image_array(sample_paths)

In [ ]:
# plot image grid
fig = px.imshow(sample_images, facet_col=0, binary_string=True, facet_row_spacing=0.0, facet_col_spacing=0)
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

# add species name and individual_id
for i, a in enumerate(fig.layout.annotations):
    a.text = samples.iloc[i].id
fig.update_layout(title=f"Random Samples", autosize=True, hovermode=False, height=400, width=1200, margin=dict(l=40, r=0, t=140, b=80))
fig.show()

<div class="alert alert-block alert-warning">🏗️ Work in progress! 👷 🚧</div> 